# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
%matplotlib inline
import matplotlib.pyplot as plt
import nltk
#nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sqlalchemy import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///CleanDF.db')
df = pd.read_sql_table("MessageCategories", con=engine)

X = df["message"]
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
moc = MultiOutputClassifier(RandomForestClassifier(n_estimators=100))

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train.values, y_train.values)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def get_report(y_test, y_pred):
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [7]:
get_report(y_test, y_pred)

related
              precision    recall  f1-score   support

           0       0.76      0.26      0.39      1588
           1       0.80      0.97      0.88      4913
           2       0.45      0.17      0.25        53

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.67      0.47      0.50      6554
weighted avg       0.78      0.79      0.75      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5474
           1       0.89      0.47      0.61      1080

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.90      0.73      0.78      6554
weighted avg       0.90      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\ma501v\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6434
           1       0.00      0.00      0.00       120

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97      6554

military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6354
           1       0.86      0.03      0.06       200

   micro avg       0.97      0.97      0.97      6554
   macro avg       0.91      0.51      0.52      6554
weighted avg       0.97      0.97      0.96      6554

child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

   micro avg       1.00      1.00      1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

water
              precision    recall  f1-score   

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001AFACD701E0>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
               oob_score=False,

In [9]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, parameters, cv=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:

cv.fit(X_train.values, y_train.values)
y_pred = cv.predict(X_test)
results = get_report(y_test, y_pred)
results

related
              precision    recall  f1-score   support

           0       0.79      0.16      0.27      1588
           1       0.78      0.99      0.87      4913
           2       0.50      0.02      0.04        53

   micro avg       0.78      0.78      0.78      6554
   macro avg       0.69      0.39      0.39      6554
weighted avg       0.78      0.78      0.72      6554

request
              precision    recall  f1-score   support

           0       0.88      1.00      0.93      5474
           1       0.93      0.30      0.45      1080

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.90      0.65      0.69      6554
weighted avg       0.89      0.88      0.85      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\ma501v\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6228
           1       1.00      0.01      0.01       326

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.98      0.50      0.49      6554
weighted avg       0.95      0.95      0.93      6554

electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6431
           1       0.00      0.00      0.00       123

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97      6554

tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6514
           1       0.00      0.00      0.00        40

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

h

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001AFACD701E0>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=None))],
 'vect': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range

In [13]:
parameters = {
    'vect__smooth_idf': [True,False],
}
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1, cv=3)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)   
results = get_report(y_test, y_pred)
results

related
              precision    recall  f1-score   support

           0       0.58      0.30      0.39      1588
           1       0.80      0.93      0.86      4913
           2       0.57      0.15      0.24        53

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.65      0.46      0.50      6554
weighted avg       0.74      0.77      0.74      6554

request
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      5474
           1       0.79      0.59      0.67      1080

   micro avg       0.91      0.91      0.91      6554
   macro avg       0.86      0.78      0.81      6554
weighted avg       0.90      0.91      0.90      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      655

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       0.36      0.10      0.16        80

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.68      0.55      0.58      6554
weighted avg       0.98      0.99      0.98      6554

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6301
           1       0.32      0.08      0.13       253

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.64      0.54      0.56      6554
weighted avg       0.94      0.96      0.95      6554

weather_related
              precision    recall  f1-score   support

           0       0.88      0.95      0.91      4711
           1       0.85      0.65      0.74      1843

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.86      0.80      0.83      6554
weighted avg       0.87      0.87    

### 9. Export your model as a pickle file

In [14]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.